# Summarise MOTBX resources

Convert MOTBX resources (YAML files) to CSV file.

In [ ]:
import os
import csv
from motbxtools import motbxschema
import json 
import pprint
pp = pprint.PrettyPrinter(indent = 2)


home_dir = "../"
schema_path = os.path.join(home_dir, "schema", "motbxschema.json")
resource_dir = os.path.join(home_dir, "resources/curated")

test_resource_dir = os.path.join(home_dir, "tests/resources_pass")
summary_csv = os.path.join(home_dir, "tests", "resources.csv")

In [ ]:
# load JSON schema from file
schema = motbxschema.MotbxSchema(schema_path)

# print schema formatted as JSON
print(json.dumps(schema.schema, indent = 2))

In [ ]:
# explore schema
print(schema.schema["properties"].keys())
print(schema.schema["properties"]["resource"]["properties"].keys())
print(schema.schema["properties"]["resourceMetadata"]["properties"].keys())

In [ ]:
# define CSV column names
fieldnames = ["resourceID"] + list(schema.schema["properties"]["resource"]["properties"].keys())
print(fieldnames)

In [ ]:
# open CSV file
with open(summary_csv, "w", newline = "", encoding = "utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    
    # iterate through resources
    for root, dirs, files in os.walk(test_resource_dir):
        for name in files: 
            if not name.endswith(".yaml"):
                continue
            print(name)
            
            # load test resource and validate
            resource = motbxschema.MotbxResource(os.path.join(root, name))
            resource.validate(schema)

            # write to CSV file
            row = resource.flatten(fieldnames) 
            pp.pprint(row)
            writer.writerow(row)